In [1]:
import time

# from qiskit.opflow import I, Z, X, Y, SparsePauliOp
import numpy as np

# 假设有 3 台机器和 3 个操作
m = 3  # 机器数量
n = 1  # 操作数量
l = 2  # 用于表示 C_max 的二进制位数

# 处理时间矩阵 (例如)
p = np.array([[1, 2, 3]])

# 常数
K1 = 10
K2 = 5

# 量子比特数量
num_qubits = m * n + l + m * l

In [2]:
#
def pauli_z_term(index, num_qubits):
    """初始化一个 Pauli-I 门的列表,指定索引 index 替换为 Pauli-Z 门"""
    paulis = ['I'] * num_qubits
    paulis[index] = 'Z'
    return ''.join(paulis)  # 将列表转化为字符串

$$\frac{1}{2} \sum_{h=0}^{l-1} 2^h - \frac{1}{2} \sum_{h=0}^{l-1} 2^h Z_{z_h}$$

In [3]:
from qiskit.quantum_info import Pauli, SparsePauliOp

# 常数项
constant_term = 1/2 * sum(2**h for h in range(l))

# 构建 Pauli-Z 项
z_terms = []
for h in range(l):
    coef = - 2**h / 2
    z_term = pauli_z_term(m * n + h, num_qubits)
    z_terms.append((coef, z_term))

# 从z_terms中提取Pauli字符串和系数
pauli_strings = [term[1] for term in z_terms]
coefficients = [term[0] for term in z_terms]
# 添加常数项
pauli_strings.append('I' * num_qubits)
coefficients.append(constant_term)
# 转换为 SparsePauliOp
paulis = [Pauli(label) for label in pauli_strings]
hamiltonian1 = SparsePauliOp.from_list(list(zip(pauli_strings, coefficients)))

print(hamiltonian1)

SparsePauliOp(['IIIZIIIIIII', 'IIIIZIIIIII', 'IIIIIIIIIII'],
              coeffs=[-0.5+0.j, -1. +0.j,  1.5+0.j])


$$K1\sum_{i=1}^{n}\left( \left( \frac{m}{2} - 1 \right)^2 -  \left( \frac{m}{2} - 1 \right)  \sum_{j=1}^{m} Z_{x_{ij}} + \frac{1}{4} \left( \sum_{j=1}^{m} Z_{x_{ij}} \right)^2\right)$$

In [4]:
from qiskit.quantum_info import SparsePauliOp, Pauli

pauli_strings = []
cofficients = []
# 常数项
constant_term1 = K1 * n * ((m / 2) - 1)**2
pauli_strings.append('I'*num_qubits)
cofficients.append(constant_term1)

# 线性项
# linear_terms = []
coef_linear = -1 * K1 * (m / 2 - 1)
for i in range(n):
    for j in range(m):
        x_term = pauli_z_term(i * m + j, num_qubits)
        pauli_strings.append(x_term)
        cofficients.append(coef_linear)
        # linear_terms.append((coef, x_term))

# 二次项
# quadratic_terms = []
coef_quadratic = K1 / 4
for i in range(n):
    for j1 in range(m):
        for j2 in range(m):
            x_term1 = pauli_z_term(i * m + j1, num_qubits)
            x_term2 = pauli_z_term(i * m + j2, num_qubits)
            if j1!=j2:
                # quadratic_terms.append((coef_quadratic, x_term1))
                # pauli_strings.append('I'*num_qubits)
                # cofficients.append(coef_quadratic)
            # else: # 如果两个pauli-z 字符串相同，结果为 I 字符串，可忽略
                combined_op = ''.join(['I' if c1 == 'I' and c2 == 'I' else 'Z' if c1 == 'Z' or c2 == 'Z' else 'I'
                                           for c1, c2 in zip(x_term1, x_term2)])
                pauli_strings.append(combined_op)
                cofficients.append(coef_quadratic)
            # quadratic_terms.append((coef, x_term1 + x_term2)) # 二次项的具体形式应该是每对 Zij 和 Zik 的组合

# 组合所有项
# pauli_strings = [term[1] for term in linear_terms + quadratic_terms]
# coefficients = [term[0] for term in linear_terms + quadratic_terms]

# 转换为 SparsePauliOp
paulis = [Pauli(label) for label in pauli_strings]
hamiltonian2 = SparsePauliOp.from_list(list(zip(pauli_strings, cofficients)))

print(hamiltonian2)

SparsePauliOp(['IIIIIIIIIII', 'ZIIIIIIIIII', 'IZIIIIIIIII', 'IIZIIIIIIII', 'ZZIIIIIIIII', 'ZIZIIIIIIII', 'ZZIIIIIIIII', 'IZZIIIIIIII', 'ZIZIIIIIIII', 'IZZIIIIIIII'],
              coeffs=[ 2.5+0.j, -5. +0.j, -5. +0.j, -5. +0.j,  2.5+0.j,  2.5+0.j,  2.5+0.j,
  2.5+0.j,  2.5+0.j,  2.5+0.j])


$$K2\sum_{j=1}^{m} \left( \frac{1}{2} \sum_{i=1}^{n} p_{ij} - \frac{1}{2} \sum_{i=1}^{n} p_{ij} Z_{x_{ij}} + \frac{1}{2} \sum_{h=0}^{l-1} 2^h Z_{z_h} - \frac{1}{2} \sum_{h=0}^{l-1} 2^h Z{\tau_{jh}} \right)^2$$

$$K2\sum_{j=1}^{m} \left( \frac{1}{2} \sum_{i=1}^{n}p_{ij}\left(I -  Z_{x_{ij}}\right) + \frac{1}{2} \sum_{h=0}^{l-1} 2^h\left( Z_{z_h} -  Z{\tau_{jh}}\right) \right)^2$$

In [5]:
# 初始化数组
pauli_strings = []
coefficients = []
# 线性和二次项
for j in range(m):
    # constant = np.sum(p[:, j]) / 2
    # constant_term = ('I', num_qubits)
    h_terms = []
    # h_terms.append((constant, constant_term))
    for i in range(n):
        coef = -p[i, j] / 2
        constant_term = ('I', num_qubits)
        z_term = pauli_z_term(i * m + j, num_qubits)
        h_terms.append((coef, constant_term))
        h_terms.append((coef, z_term))

    for h in range(l):
        coef = 2 ** h / 2
        tau_term = pauli_z_term(m * n + j * l + h, num_qubits)
        z_term = pauli_z_term(m * n + h, num_qubits)
        h_terms.append((coef, z_term))
        h_terms.append((-coef, tau_term))

    # 平方项展开
    # pauli_strings.append('I' * num_qubits)
    # coefficients.append(K2 * constant_term ** 2)

    for term1, op1 in h_terms:
        # pauli_strings.append(op1)
        # coefficients.append(K2 * term1 ** 2)
        for term2, op2 in h_terms:
            if op1 != op2:
                combined_op = ''.join(['I' if (c1 == 'I' and c2 == 'I') or (c1 == 'Z' and c2 == 'Z') else 'Z'  for c1, c2 in zip(op1, op2)])
                pauli_strings.append(combined_op)
                coefficients.append(2 * K2 * term1 * term2)

# 转换为 SparsePauliOp
paulis = [Pauli(label) for label in pauli_strings]
hamiltonian3 = SparsePauliOp.from_list(list(zip(pauli_strings, coefficients)))
# hamiltonian = PauliSumOp.from_list(list(zip(pauli_strings, coefficients)))

print(hamiltonian3)


SparsePauliOp(['ZZ', 'IZ', 'IZ', 'IZ', 'IZ', 'ZZ', 'ZI', 'ZI', 'ZI', 'ZI', 'IZ', 'ZI', 'II', 'II', 'IZ', 'ZI', 'II', 'II', 'IZ', 'ZI', 'II', 'II', 'IZ', 'ZI', 'II', 'II', 'IZ', 'IZ', 'IZ', 'IZ', 'IZ', 'IZ', 'IZ', 'IZ', 'IZ', 'IZ', 'IZ', 'IZ', 'II', 'II', 'II', 'IZ', 'IZ', 'II', 'II', 'II', 'IZ', 'IZ', 'II', 'II', 'II', 'IZ', 'IZ', 'II', 'II', 'II', 'IZ', 'IZ', 'IZ', 'IZ', 'IZ', 'IZ', 'II', 'II', 'II', 'II', 'IZ', 'II', 'II', 'II', 'II', 'IZ', 'II', 'II', 'II', 'II', 'IZ', 'II', 'II', 'II', 'II', 'IZ', 'II', 'II', 'II', 'II'],
              coeffs=[  2.5+0.j,  -2.5+0.j,   2.5+0.j,  -5. +0.j,   5. +0.j,   2.5+0.j,
  -2.5+0.j,   2.5+0.j,  -5. +0.j,   5. +0.j,  -2.5+0.j,  -2.5+0.j,
   5. +0.j,  -5. +0.j,   2.5+0.j,   2.5+0.j,  -5. +0.j,   5. +0.j,
  -5. +0.j,  -5. +0.j,   5. +0.j,  -5. +0.j,   5. +0.j,   5. +0.j,
  -5. +0.j,   5. +0.j,  10. +0.j,  -5. +0.j,   5. +0.j, -10. +0.j,
  10. +0.j,  10. +0.j,  -5. +0.j,   5. +0.j, -10. +0.j,  10. +0.j,
  -5. +0.j,  -5. +0.j,  -2.5+0.j,   5. +0.j, 

$$K2\sum_{j=1}^{m} \left( \frac{1}{2} \sum_{i=1}^{n} p_{ij} - \frac{1}{2} \sum_{i=1}^{n} p_{ij} Z_{x_{ij}} + \frac{1}{2} \sum_{h=0}^{l-1} 2^h Z_{z_h} - \frac{1}{2} \sum_{h=0}^{l-1} 2^h Z{\tau_{jh}} \right)^2$$

In [20]:
hamiltonian =  hamiltonian2
print(hamiltonian)

SparsePauliOp(['ZIIIIIIIIII', 'IZIIIIIIIII', 'IIZIIIIIIII', 'ZZIIIIIIIII', 'ZIZIIIIIIII', 'ZZIIIIIIIII', 'IZZIIIIIIII', 'ZIZIIIIIIII', 'IZZIIIIIIII'],
              coeffs=[-5. +0.j, -5. +0.j, -5. +0.j,  2.5+0.j,  2.5+0.j,  2.5+0.j,  2.5+0.j,
  2.5+0.j,  2.5+0.j])


In [21]:
from qiskit import QuantumCircuit

def create_qaoa_circuit(H, betas, gammas, num_qubits):
    qc = QuantumCircuit(num_qubits)
    # 初始化到 superposition
    qc.h(range(num_qubits))

    # Cost Hamiltonian
    for beta, gamma in zip(betas, gammas):
        for term in H:
            pauli_string = term.primitive.to_label()
            coeff = term.coeff.real
            if pauli_string.count('Z') == 1:
                idx = pauli_string.index('Z')
                qc.rz(2 * gamma * coeff, idx)
            elif pauli_string.count('Z') == 2:
                idxs = [i for i, p in enumerate(pauli_string) if p == 'Z']
                qc.rzz(2 * gamma * coeff, idxs[0], idxs[1])

        # Mixer Hamiltonian
        qc.rx(2 * beta, range(num_qubits))

    return qc

In [22]:
from qiskit.opflow import PauliSumOp


# 定义混合哈密顿量 H_M
def get_mixer_hamiltonian(num_qubits):
    pauli_list = []
    coeffs = []
    for i in range(num_qubits):
        x_term = ['I'] * num_qubits
        x_term[i] = 'X'
        pauli_str = ''.join(x_term)
        pauli_list.append((pauli_str, 1.0))
    mixer_hamiltonian = PauliSumOp.from_list(pauli_list)
    return mixer_hamiltonian

mixer_hamiltonian = get_mixer_hamiltonian(num_qubits)
print(mixer_hamiltonian)

1.0 * XIIIIIIIIII
+ 1.0 * IXIIIIIIIII
+ 1.0 * IIXIIIIIIII
+ 1.0 * IIIXIIIIIII
+ 1.0 * IIIIXIIIIII
+ 1.0 * IIIIIXIIIII
+ 1.0 * IIIIIIXIIII
+ 1.0 * IIIIIIIXIII
+ 1.0 * IIIIIIIIXII
+ 1.0 * IIIIIIIIIXI
+ 1.0 * IIIIIIIIIIX


In [23]:
from qiskit.primitives import Estimator
estimator = Estimator()
def objective_function(params,ansatz):
    bound_circuit = ansatz.bind_parameters(params)
    bound_circuit = transpile(bound_circuit, Aer.get_backend('aer_simulator'))
    result = estimator.run([bound_circuit], [hamiltonian]).result()
    expectation_value = result.values[0]
    return expectation_value

In [24]:
from functools import partial
from qiskit.circuit import ParameterVector
from qiskit.circuit.library import PauliEvolutionGate
from qiskit import transpile, Aer,execute
from qiskit.primitives import Estimator
from qiskit_algorithms.optimizers import COBYLA
import matplotlib.pyplot as plt

P = 3

startTime = time.process_time()
ansatz_final = object()
init_point = np.array([])
counts = {}
min_energy = 0
for p in range(1, P + 1):
    min_energy = 10000
    for k in range(50):
        print('第：', p, '层', ' ', '第', k, '个参数')
        init_point_temp = init_point
        beta_p = np.random.uniform(0, 2 * np.pi)
        gamma_p = np.random.uniform(0,  2 * np.pi)
        if k >0:
            init_point_temp = np.delete(init_point_temp, 2 * p - 1)
            init_point_temp = np.delete(init_point_temp, p - 1)
        init_point_temp = np.insert(init_point_temp, p - 1, beta_p)
        init_point_temp = np.insert(init_point_temp, 2 * p - 1, gamma_p)
        # 创建参数向量
        params = ParameterVector('theta',2 * p)
        ansatz = QuantumCircuit(num_qubits)
        ansatz.h(range(num_qubits))
        for layer in range(p):
            # 应用问题哈密顿量
            ansatz.append(PauliEvolutionGate(hamiltonian, params[2 * layer + 1]), list(range(num_qubits)))
            # 应用混合哈密顿量
            ansatz.append(PauliEvolutionGate(mixer_hamiltonian, params[2 * layer]), list(range(num_qubits)))
        # 定义目标函数
        optimizer = COBYLA(maxiter=10000,disp=True)
        # 使用lambda函数传递额外参数
        objective_function_params = partial(objective_function, ansatz = ansatz)

        result = optimizer.minimize(fun=objective_function_params, x0=init_point_temp)

        # print(result.x)
        optimal_params = result.x
        # 量子电路测量
        bound_circuit = ansatz.bind_parameters(optimal_params)
        bound_circuit.measure_all()

        # 运行电路并获取测量结果
        simulator = Aer.get_backend('qasm_simulator')
        tqc = transpile(bound_circuit, simulator)
        result = execute(tqc, backend=simulator, shots=10000).result()
        counts_temp = result.get_counts()

        # 计算最终的能量
        energy = objective_function(optimal_params,ansatz)
        # 寻找这20对点中最优的参数
        if min_energy > energy:
            min_energy = energy
            init_point = init_point_temp
            counts = counts_temp
            ansatz_final = ansatz

endTime = time.process_time()
print("最小能量：",min_energy)
sorted_dict = dict(sorted(counts.items(), key=lambda item: item[1], reverse=True))

print('优化时间：{}s'.format(endTime - startTime))
print(sorted_dict)
print(init_point)
ansatz_final.decompose().draw(output='mpl', style="iqp")
plt.show()



第： 1 层   第 0 个参数


C:\Users\18330\AppData\Local\Temp\ipykernel_21024\1959600083.py:4: DeprecationWarning: The method ``qiskit.circuit.quantumcircuit.QuantumCircuit.bind_parameters()`` is deprecated as of qiskit 0.45.0. It will be removed no earlier than 3 months after the release date. Use assign_parameters() instead
  bound_circuit = ansatz.bind_parameters(params)
C:\Users\18330\AppData\Local\Temp\ipykernel_21024\3037650019.py:47: DeprecationWarning: The method ``qiskit.circuit.quantumcircuit.QuantumCircuit.bind_parameters()`` is deprecated as of qiskit 0.45.0. It will be removed no earlier than 3 months after the release date. Use assign_parameters() instead
  bound_circuit = ansatz.bind_parameters(optimal_params)


第： 1 层   第 1 个参数
第： 1 层   第 2 个参数
第： 1 层   第 3 个参数
第： 1 层   第 4 个参数
第： 1 层   第 5 个参数
第： 1 层   第 6 个参数
第： 1 层   第 7 个参数
第： 1 层   第 8 个参数
第： 1 层   第 9 个参数
第： 1 层   第 10 个参数
第： 1 层   第 11 个参数
第： 1 层   第 12 个参数
第： 1 层   第 13 个参数
第： 1 层   第 14 个参数
第： 1 层   第 15 个参数
第： 1 层   第 16 个参数
第： 1 层   第 17 个参数
第： 1 层   第 18 个参数
第： 1 层   第 19 个参数
第： 1 层   第 20 个参数
第： 1 层   第 21 个参数
第： 1 层   第 22 个参数
第： 1 层   第 23 个参数
第： 1 层   第 24 个参数
第： 1 层   第 25 个参数
第： 1 层   第 26 个参数
第： 1 层   第 27 个参数
第： 1 层   第 28 个参数
第： 1 层   第 29 个参数
第： 1 层   第 30 个参数
第： 1 层   第 31 个参数
第： 1 层   第 32 个参数
第： 1 层   第 33 个参数
第： 1 层   第 34 个参数
第： 1 层   第 35 个参数
第： 1 层   第 36 个参数
第： 1 层   第 37 个参数
第： 1 层   第 38 个参数
第： 1 层   第 39 个参数
第： 1 层   第 40 个参数
第： 1 层   第 41 个参数
第： 1 层   第 42 个参数
第： 1 层   第 43 个参数
第： 1 层   第 44 个参数
第： 1 层   第 45 个参数
第： 1 层   第 46 个参数
第： 1 层   第 47 个参数
第： 1 层   第 48 个参数
第： 1 层   第 49 个参数
第： 2 层   第 0 个参数
第： 2 层   第 1 个参数
第： 2 层   第 2 个参数
第： 2 层   第 3 个参数
第： 2 层   第 4 个参数
第： 2 层   第 5 个参数
第： 2 层   第 6 个参数
第： 2 层  

In [11]:
from functools import partial
from qiskit.circuit import ParameterVector
from qiskit.circuit.library import PauliEvolutionGate
from qiskit import transpile, Aer,execute
from qiskit.primitives import Estimator
from qiskit_algorithms.optimizers import COBYLA
import matplotlib.pyplot as plt

P = 6

startTime = time.process_time()
ansatz_final = object()
init_point = np.array([])
counts = {}
min_energy = 1000
for k in range(10):
    print( '第', k, '次优化')

    point = np.random.uniform(0,2*np.pi,2*P)
    # 创建参数向量
    params = ParameterVector('theta',2 * P)
    ansatz = QuantumCircuit(num_qubits)
    for layer in range(P):
        # 应用问题哈密顿量
        ansatz.append(PauliEvolutionGate(hamiltonian, params[2 * layer + 1]), list(range(num_qubits)))
        # 应用混合哈密顿量
        ansatz.append(PauliEvolutionGate(mixer_hamiltonian, params[2 * layer]), list(range(num_qubits)))
        # 定义目标函数
    optimizer = COBYLA(maxiter=10000,disp=True)
    # 使用lambda函数传递额外参数
    objective_function_params = partial(objective_function, ansatz = ansatz)

    result = optimizer.minimize(fun=objective_function_params, x0=point)

    # print(result.x)
    optimal_params = result.x
    # 量子电路测量
    bound_circuit = ansatz.bind_parameters(optimal_params)
    bound_circuit.measure_all()

    # 运行电路并获取测量结果
    simulator = Aer.get_backend('qasm_simulator')
    tqc = transpile(bound_circuit, simulator)
    result = execute(tqc, backend=simulator, shots=10000).result()
    counts_temp = result.get_counts()

    # 计算最终的能量
    energy = objective_function(optimal_params,ansatz)
    # 寻找这20对点中最优的参数
    if min_energy > energy:
        min_energy = energy
        counts = counts_temp
        ansatz_final = ansatz

endTime = time.process_time()
print("最小能量：",min_energy)
sorted_dict = dict(sorted(counts.items(), key=lambda item: item[1], reverse=True))

print('优化时间：{}s'.format(endTime - startTime))
print(sorted_dict)
# print(init_point)

第 0 次优化


C:\Users\18330\AppData\Local\Temp\ipykernel_21024\1959600083.py:4: DeprecationWarning: The method ``qiskit.circuit.quantumcircuit.QuantumCircuit.bind_parameters()`` is deprecated as of qiskit 0.45.0. It will be removed no earlier than 3 months after the release date. Use assign_parameters() instead
  bound_circuit = ansatz.bind_parameters(params)
C:\Users\18330\AppData\Local\Temp\ipykernel_21024\3090426580.py:38: DeprecationWarning: The method ``qiskit.circuit.quantumcircuit.QuantumCircuit.bind_parameters()`` is deprecated as of qiskit 0.45.0. It will be removed no earlier than 3 months after the release date. Use assign_parameters() instead
  bound_circuit = ansatz.bind_parameters(optimal_params)


第 1 次优化
第 2 次优化
第 3 次优化
第 4 次优化
第 5 次优化
第 6 次优化
第 7 次优化
第 8 次优化
第 9 次优化
最小能量： 9.474701331444943e-08
优化时间：156.6875s
{'00111111': 791, '11111111': 788, '01111111': 754, '10111111': 749, '00101111': 199, '11111110': 197, '10111101': 188, '00011111': 184, '00111110': 180, '00111101': 180, '11110111': 178, '10111011': 177, '00110111': 173, '01111011': 172, '11111101': 170, '10101111': 170, '10110111': 169, '10011111': 167, '11111011': 167, '01101111': 166, '01110111': 166, '00111011': 166, '10111110': 159, '11101111': 156, '01111110': 155, '01011111': 152, '01111101': 148, '11011111': 141, '11111100': 55, '11101101': 46, '01111100': 46, '01010111': 42, '11011011': 42, '10110011': 42, '00101011': 42, '10110101': 42, '01110101': 40, '01011101': 40, '00101101': 39, '11011101': 39, '10111100': 39, '10111010': 39, '11011110': 39, '00110110': 39, '11110011': 38, '10011011': 38, '00010111': 38, '01111001': 38, '10100111': 36, '00111100': 36, '01101011': 36, '10101011': 36, '01110011': 35, '0110110

In [13]:
# 测量结果
# measurement_result = '000011110100'
measurement_result = '11111111'
# 将测量结果转换为+1和-1
def measurement_to_expectation(measurement):
    return np.array([-1 if bit == '1' else 1 for bit in measurement])

# 计算每个项的期望值并累加
def calculate_energy(hamiltonian, measurement_result):
    energy = 0.0
    expectation_values = measurement_to_expectation(measurement_result)

    for pauli, coeff in zip(hamiltonian.paulis.to_labels(), hamiltonian.coeffs):
        term_value = 1.0
        for idx, pauli_char in enumerate(pauli):
            if pauli_char == 'Z':
                term_value *= expectation_values[idx]
        energy += coeff * term_value

    return energy

# print(hamiltonian)
# 计算并输出能量
energy = calculate_energy(hamiltonian, measurement_result)
print(f"Energy for measurement result {measurement_result}: {energy}")

Energy for measurement result 11111111: (5+0j)
